In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Instalar as dependencias

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz

In [ ]:
!tar xf spark-2.4.8-bin-hadoop2.7.tgz

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"

# Criar uma sessao Spark

In [ ]:
import findspark
# PySpark isn't on sys.path by default
# Findspark adds pyspark to sys.path at runtime in order to locate spark in the system
findspark.init()
from pyspark.sql import SparkSession
# local session
spark = SparkSession.builder.master("local[*]").config('spark.ui.port', '4050').getOrCreate()
spark

# Configuraçao do Spark UI

In [ ]:
# ngrok is a cross-platform application that enables developers to expose a local development server to the Internet with minimal effort
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2021-10-08 22:32:15--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.202.168.65, 18.205.222.128, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.202.168.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.2’

ngrok-stable-linux- 100%[===================>]  13.19M  --.-KB/s    in 0.1s    

2021-10-08 22:32:16 (97.0 MB/s) - ‘ngrok-stable-linux-amd64.zip.2’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [ ]:
get_ipython().system_raw('./ngrok http 4050 &')

In [ ]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://badd-34-86-242-50.ngrok.io","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://badd-34-86-242-50.ngrok.io","proto":"http","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


Agora você deve ser capaz de visualizar os trabalhos e suas etapas no link criado "public_url".

# Ler um dataset

## Exemplo simples

In [ ]:
l = [('Alice', 1)]

In [ ]:
df = spark.createDataFrame(l, ['name', 'age'])
df.collect()

In [ ]:
df.explain()

In [ ]:
df.registerTempTable("people")
df2 = spark.sql("select * from people")

In [ ]:
df2.collect()

In [ ]:
spark.catalog.dropTempView("people")

## Outro Exemplo

**crim**
per capita crime rate by town.

**zn**
proportion of residential land zoned for lots over 25,000 sq.ft.

**indus**
proportion of non-retail business acres per town.

**chas**
Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).

**nox**
nitrogen oxides concentration (parts per 10 million).

**rm**
average number of rooms per dwelling.

**age**
proportion of owner-occupied units built prior to 1940.

**dis**
weighted mean of distances to five Boston employment centres.

**rad**
index of accessibility to radial highways.

**tax**
full-value property-tax rate per \$10,000.

**ptratio**
pupil-teacher ratio by town.

**black**
1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town.

**lstat**
lower status of the population (percent).

**medv**
median value of owner-occupied homes in \$1000s.


In [ ]:
from google.colab import files
files.upload()
!ls

Saving word_count_ex.txt to word_count_ex.txt
ngrok			      spark-2.4.8-bin-hadoop2.7
ngrok-stable-linux-amd64.zip  spark-2.4.8-bin-hadoop2.7.tgz
sample_data		      word_count_ex.txt


In [ ]:
df = spark.read.csv('BostonHousing.csv',inferSchema=True, header =True)

In [ ]:
df.printSchema()

root
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: integer (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: integer (nullable = true)
 |-- tax: integer (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- b: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)



In [ ]:
df.registerTempTable("boston")

In [ ]:
df.summary().show()

+-------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|summary|              crim|                zn|             indus|              chas|                nox|                rm|               age|              dis|              rad|               tax|           ptratio|                 b|             lstat|              medv|
+-------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|  count|               506|               506|               506|               506|                506|               506|               506|              506|              

In [ ]:
df2 = df.where("age < 20").groupBy("age").avg("crim").orderBy(df.age.asc())
df2.show()

+----+---------+
| age|avg(crim)|
+----+---------+
| 2.9|  0.12744|
| 6.0|  0.07896|
| 6.2|  0.19539|
| 6.5|  0.15936|
| 6.6|  0.11488|
| 6.8|  0.08221|
| 7.8| 0.109605|
| 8.4|  0.36894|
| 8.9|  0.21409|
| 9.8|  0.28955|
| 9.9|  0.02187|
|10.0|  0.05561|
|13.0|   0.1403|
|13.9|  0.01778|
|14.7|  0.19186|
|15.3|   0.0315|
|15.7|  0.02177|
|15.8|  0.03359|
|16.3|  0.16211|
|17.0|  0.29819|
+----+---------+
only showing top 20 rows



In [ ]:
df2 = spark.sql("select age, avg(crim) from boston where age < 20 group by age order by age asc")
df2.show()

+----+---------+
| age|avg(crim)|
+----+---------+
| 2.9|  0.12744|
| 6.0|  0.07896|
| 6.2|  0.19539|
| 6.5|  0.15936|
| 6.6|  0.11488|
| 6.8|  0.08221|
| 7.8| 0.109605|
| 8.4|  0.36894|
| 8.9|  0.21409|
| 9.8|  0.28955|
| 9.9|  0.02187|
|10.0|  0.05561|
|13.0|   0.1403|
|13.9|  0.01778|
|14.7|  0.19186|
|15.3|   0.0315|
|15.7|  0.02177|
|15.8|  0.03359|
|16.3|  0.16211|
|17.0|  0.29819|
+----+---------+
only showing top 20 rows



In [ ]:
df2.explain()

== Physical Plan ==
*(3) Sort [age#38 ASC NULLS FIRST], true, 0
+- Exchange rangepartitioning(age#38 ASC NULLS FIRST, 200)
   +- *(2) HashAggregate(keys=[age#38], functions=[avg(crim#32)])
      +- Exchange hashpartitioning(age#38, 200)
         +- *(1) HashAggregate(keys=[age#38], functions=[partial_avg(crim#32)])
            +- *(1) Project [crim#32, age#38]
               +- *(1) Filter (isnotnull(age#38) && (age#38 < 20.0))
                  +- *(1) FileScan csv [crim#32,age#38] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/content/BostonHousing.csv], PartitionFilters: [], PushedFilters: [IsNotNull(age), LessThan(age,20.0)], ReadSchema: struct<crim:double,age:double>


# Exemplo Pratico - Como Funciona?

**parallelize**: Distribute a local Python collection to form an RDD. Using xrange is recommended if the input represents a range for performance

In [ ]:
sc = spark.sparkContext

## Word count

In [ ]:
text_file = sc.textFile("word_count_ex.txt")
counts = text_file.flatMap(lambda line: line.split(" ")) \
             .map(lambda word: (word, 1)) \
             .reduceByKey(lambda a, b: a + b)

In [ ]:
counts.collect()

[('Aine', 3), ('Amy', 1), ('Iwona', 4), ('Canan', 2)]

In [ ]:
counts.getNumPartitions()

2

In [ ]:
counts.saveAsTextFile("res_word_count_ex.txt")

In [ ]:
print(counts.toDebugString().decode())

(2) PythonRDD[30] at RDD at PythonRDD.scala:53 []
 |  MapPartitionsRDD[26] at mapPartitions at PythonRDD.scala:133 []
 |  ShuffledRDD[25] at partitionBy at NativeMethodAccessorImpl.java:0 []
 +-(2) PairwiseRDD[24] at reduceByKey at <ipython-input-21-23e481d86be9>:2 []
    |  PythonRDD[23] at reduceByKey at <ipython-input-21-23e481d86be9>:2 []
    |  word_count_ex.txt MapPartitionsRDD[22] at textFile at NativeMethodAccessorImpl.java:0 []
    |  word_count_ex.txt HadoopRDD[21] at textFile at NativeMethodAccessorImpl.java:0 []


## Transformaçoes a Açoes

In [ ]:
# map
rdd = sc.parallelize(["b", "a", "c"])
sorted(rdd.map(lambda x: (x, 1)).collect())

In [ ]:
# filter
rdd = sc.parallelize([1, 2, 3, 4, 5])
rdd.filter(lambda x: x % 2 == 0).collect()

sc.parallelize(range(100), 100).filter(lambda x: x > 90).take(3)

In [ ]:
# Açoes
# reduce
from operator import add
sc.parallelize([1, 2, 3, 4, 5]).reduce(add)

In [ ]:
# reduce
sc.parallelize((2 for _ in range(10))).map(lambda x: 1).cache().reduce(add)

# count
sc.parallelize([2, 3, 4]).count()

# collect
sc.parallelize([1, 2, 3, 4, 5], 3).glom().collect()

In [ ]:
# TODO
# https://spark.apache.org/docs/2.4.0/api/python/pyspark.html

# Job, Stage, Task

- O RDD é uma coleçao de partiçoes.  
- Cada core executa uma **task** numa partiçao.
- Um **job** e representado por um DAG (Directed Acyclic Graph)
- Um DAG é composto por um ou varias **stages**
  - as separaçoes dos **stages** correspondem a uma necessidade de movimentar os dados entre as partiçoes (shuffle)
  - dentro de um stage, as transformaçoes vao ser paralelizadas entre as **tasks**

In [ ]:
# https://spark.apache.org/docs/2.4.0/api/python/pyspark.html

In [ ]:
sc = spark.sparkContext

In [ ]:
# filter
rdd = sc.parallelize(range(10000))

In [ ]:
rdd.count()

10000

In [ ]:
# stage1
rdd2 = (rdd.filter(lambda x: x < 1000.0)
  .map(lambda x: (x, x) )
# stage2
  .groupByKey()
  .map( lambda x: (sum(x[1]), x[0]))
# stage 3
  .sortByKey()
  .count())

## Com Spark SQL

In [ ]:
from google.colab import files
files.upload()
!ls

Saving BostonHousing.csv to BostonHousing.csv
BostonHousing.csv  ngrok-stable-linux-amd64.zip  spark-2.4.8-bin-hadoop2.7
ngrok		   sample_data			 spark-2.4.8-bin-hadoop2.7.tgz


In [ ]:
df = spark.read.csv('BostonHousing.csv',inferSchema=True, header =True)
df.summary().show()

+-------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|summary|              crim|                zn|             indus|              chas|                nox|                rm|               age|              dis|              rad|               tax|           ptratio|                 b|             lstat|              medv|
+-------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|  count|               506|               506|               506|               506|                506|               506|               506|              506|              

In [ ]:
df.count()

506

In [ ]:
df.rdd.count()

506

In [ ]:
df2 = df.where("age < 20").groupBy("age").avg("crim")
df2.collect()

[Row(age=15.7, avg(crim)=0.02177),
 Row(age=6.6, avg(crim)=0.11488),
 Row(age=2.9, avg(crim)=0.12744),
 Row(age=15.8, avg(crim)=0.03359),
 Row(age=6.5, avg(crim)=0.15936),
 Row(age=17.7, avg(crim)=0.1),
 Row(age=8.4, avg(crim)=0.36894),
 Row(age=9.8, avg(crim)=0.28955),
 Row(age=17.5, avg(crim)=0.174745),
 Row(age=18.4, avg(crim)=0.08765500000000001),
 Row(age=18.5, avg(crim)=0.07744000000000001),
 Row(age=13.9, avg(crim)=0.01778),
 Row(age=19.1, avg(crim)=0.03548),
 Row(age=8.9, avg(crim)=0.21409),
 Row(age=6.2, avg(crim)=0.19539),
 Row(age=9.9, avg(crim)=0.02187),
 Row(age=14.7, avg(crim)=0.19186),
 Row(age=15.3, avg(crim)=0.0315),
 Row(age=17.2, avg(crim)=0.06724),
 Row(age=19.5, avg(crim)=0.10659),
 Row(age=17.0, avg(crim)=0.29819),
 Row(age=6.8, avg(crim)=0.08221),
 Row(age=7.8, avg(crim)=0.109605),
 Row(age=10.0, avg(crim)=0.05561),
 Row(age=13.0, avg(crim)=0.1403),
 Row(age=17.8, avg(crim)=0.03584),
 Row(age=6.0, avg(crim)=0.07896),
 Row(age=16.3, avg(crim)=0.16211),
 Row(age=18

In [ ]:
df2 = df.where("age < 20").groupBy("age").avg("crim")
df2.collect()

[Row(age=15.7, avg(crim)=0.02177),
 Row(age=6.6, avg(crim)=0.11488),
 Row(age=2.9, avg(crim)=0.12744),
 Row(age=15.8, avg(crim)=0.03359),
 Row(age=6.5, avg(crim)=0.15936),
 Row(age=17.7, avg(crim)=0.1),
 Row(age=8.4, avg(crim)=0.36894),
 Row(age=9.8, avg(crim)=0.28955),
 Row(age=17.5, avg(crim)=0.174745),
 Row(age=18.4, avg(crim)=0.08765500000000001),
 Row(age=18.5, avg(crim)=0.07744000000000001),
 Row(age=13.9, avg(crim)=0.01778),
 Row(age=19.1, avg(crim)=0.03548),
 Row(age=8.9, avg(crim)=0.21409),
 Row(age=6.2, avg(crim)=0.19539),
 Row(age=9.9, avg(crim)=0.02187),
 Row(age=14.7, avg(crim)=0.19186),
 Row(age=15.3, avg(crim)=0.0315),
 Row(age=17.2, avg(crim)=0.06724),
 Row(age=19.5, avg(crim)=0.10659),
 Row(age=17.0, avg(crim)=0.29819),
 Row(age=6.8, avg(crim)=0.08221),
 Row(age=7.8, avg(crim)=0.109605),
 Row(age=10.0, avg(crim)=0.05561),
 Row(age=13.0, avg(crim)=0.1403),
 Row(age=17.8, avg(crim)=0.03584),
 Row(age=6.0, avg(crim)=0.07896),
 Row(age=16.3, avg(crim)=0.16211),
 Row(age=18

In [ ]:
df2 = df.where("age < 20").groupBy("age").avg("crim").orderBy(df.age.asc())
df2.collect()

[Row(age=2.9, avg(crim)=0.12744),
 Row(age=6.0, avg(crim)=0.07896),
 Row(age=6.2, avg(crim)=0.19539),
 Row(age=6.5, avg(crim)=0.15936),
 Row(age=6.6, avg(crim)=0.11488),
 Row(age=6.8, avg(crim)=0.08221),
 Row(age=7.8, avg(crim)=0.109605),
 Row(age=8.4, avg(crim)=0.36894),
 Row(age=8.9, avg(crim)=0.21409),
 Row(age=9.8, avg(crim)=0.28955),
 Row(age=9.9, avg(crim)=0.02187),
 Row(age=10.0, avg(crim)=0.05561),
 Row(age=13.0, avg(crim)=0.1403),
 Row(age=13.9, avg(crim)=0.01778),
 Row(age=14.7, avg(crim)=0.19186),
 Row(age=15.3, avg(crim)=0.0315),
 Row(age=15.7, avg(crim)=0.02177),
 Row(age=15.8, avg(crim)=0.03359),
 Row(age=16.3, avg(crim)=0.16211),
 Row(age=17.0, avg(crim)=0.29819),
 Row(age=17.2, avg(crim)=0.06724),
 Row(age=17.5, avg(crim)=0.174745),
 Row(age=17.7, avg(crim)=0.1),
 Row(age=17.8, avg(crim)=0.03584),
 Row(age=18.4, avg(crim)=0.08765500000000001),
 Row(age=18.5, avg(crim)=0.07744000000000001),
 Row(age=18.8, avg(crim)=0.01439),
 Row(age=19.1, avg(crim)=0.03548),
 Row(age=19

# Spark UI

**Dica**: para testar o seu pipeline de transformaçoes,
pega uma amostra do dados em produçao, pequena o suficiente para caber na memoria do seu driver. Depois disso, executa as suas transformaçoes e monitora no **Spark UI**.

- Verifica que tem o minimo de stages possiveis
  - os estagios nao podem ser executados em paralelo (ao final de cada stage é requisito a comunicaçao com o driver)
  - so podem ser executados em paralelo se 2 stages estao calculando um RDD diferente (no caso de um join, por exemplo)
- Verifica que a distribuiçao do tempo de execuçao das tasks é uniformo

In [ ]:
from google.colab import files
files.upload()
!ls

Saving BostonHousing.csv to BostonHousing.csv
BostonHousing.csv  ngrok-stable-linux-amd64.zip  spark-2.4.8-bin-hadoop2.7
ngrok		   sample_data			 spark-2.4.8-bin-hadoop2.7.tgz


In [ ]:
df = spark.read.csv('BostonHousing.csv',inferSchema=True, header =True)
df.summary().show()

+-------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|summary|              crim|                zn|             indus|              chas|                nox|                rm|               age|              dis|              rad|               tax|           ptratio|                 b|             lstat|              medv|
+-------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|  count|               506|               506|               506|               506|                506|               506|               506|              506|              

In [ ]:
df2 = df.select("age", "crim").where("age < 77.3").groupBy("age").avg()
df2.collect()

AnalysisException: ignored

# Spark SQL

- Um exemplo de processamento com RDD
- O mesmo processamento com Spark SQL
- 2 exemplos da mesma query para analisarmos os planos de execuçao gerados pelo Spark SQL

In [ ]:
from google.colab import files
files.upload()
!ls

Saving BostonHousing.csv to BostonHousing (1).csv
'BostonHousing (1).csv'         sample_data
 BostonHousing.csv	        spark-2.4.8-bin-hadoop2.7
 ngrok			        spark-2.4.8-bin-hadoop2.7.tgz
 ngrok-stable-linux-amd64.zip


In [ ]:
df = spark.read.csv('BostonHousing.csv',inferSchema=True, header =True)
df.summary().show()

+-------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|summary|              crim|                zn|             indus|              chas|                nox|                rm|               age|              dis|              rad|               tax|           ptratio|                 b|             lstat|              medv|
+-------+------------------+------------------+------------------+------------------+-------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+------------------+
|  count|               506|               506|               506|               506|                506|               506|               506|              506|              

In [ ]:
df.registerTempTable("boston")

spark.sql("select crim, zn from boston t1 join boston t2 on t1.indus on t2.indus where t1.nox > 100")

In [ ]:
df2 = df.select("age", "crim").where("age < 77.3").groupBy("age").avg()
df2.collect()

[Row(age=74.5, avg(age)=74.5, avg(crim)=0.10153),
 Row(age=56.8, avg(age)=56.8, avg(crim)=0.07151),
 Row(age=15.7, avg(age)=15.7, avg(crim)=0.02177),
 Row(age=67.0, avg(age)=67.0, avg(crim)=0.57834),
 Row(age=56.5, avg(age)=56.5, avg(crim)=0.62739),
 Row(age=32.3, avg(age)=32.3, avg(crim)=0.14052),
 Row(age=49.9, avg(age)=49.9, avg(crim)=0.35114),
 Row(age=54.2, avg(age)=54.2, avg(crim)=0.06905),
 Row(age=65.4, avg(age)=65.4, avg(crim)=7.83932),
 Row(age=59.5, avg(age)=59.5, avg(crim)=0.01951),
 Row(age=76.5, avg(age)=76.5, avg(crim)=4.179653333333333),
 Row(age=73.5, avg(age)=73.5, avg(crim)=0.17783),
 Row(age=66.2, avg(age)=66.2, avg(crim)=0.14932),
 Row(age=71.6, avg(age)=71.6, avg(crim)=0.51183),
 Row(age=70.2, avg(age)=70.2, avg(crim)=0.19133),
 Row(age=49.7, avg(age)=49.7, avg(crim)=0.06129),
 Row(age=69.1, avg(age)=69.1, avg(crim)=0.06263),
 Row(age=31.1, avg(age)=31.1, avg(crim)=0.08655499999999999),
 Row(age=38.5, avg(age)=38.5, avg(crim)=0.03738),
 Row(age=74.4, avg(age)=74.4

## Optimization plan

In [ ]:
df1 = spark.createDataFrame([(1, "andy"), (2, "bob"), (2, "andy")], ("pets", "name"))

In [ ]:
df1.registerTempTable("table")

In [ ]:
spark.sql("select name, sum(pets) from table group by name").show()

+----+---------+
|name|sum(pets)|
+----+---------+
|andy|        3|
| bob|        2|
+----+---------+



In [ ]:
df2 = spark.createDataFrame([(4, "andy"), (2, "bob"), (6, "bob")], ("cartoes", "name"))

In [ ]:
df = df1.join(df2, "name", "inner").select("pets")
df.show() 

+----+
|pets|
+----+
|   1|
|   2|
|   2|
|   2|
+----+



In [ ]:
df1.createOrReplaceTempView("table1")
df2.createOrReplaceTempView("table2")

df = spark.sql("select pets, cartoes from table1 t1 join table2 t2 on t1.name = t2.name where t1.name = 'andy'")
df.show()

+----+-------+
|pets|cartoes|
+----+-------+
|   1|      4|
|   2|      4|
+----+-------+



In [ ]:
df = spark.sql("select pets, cartoes from (select * from table1 where name = 'andy') t1 join table2 t2 on t1.name = t2.name")
df.show()

+----+-------+
|pets|cartoes|
+----+-------+
|   1|      4|
|   2|      4|
+----+-------+



## Spark SQL na pratica

In [ ]:
transaction = spark.createDataFrame([(1, 1, 1, 200), (2, 1, 2, 100), (3, 2, 1, 300), (4, 2, 2, 500), (5, 1, 1, 30)], ("TransactionID", "ProductID", "ClientID", "TransactionValue"))
client = spark.createDataFrame([(1, "Sarah"), (2, "Joao")], ("ClientID", "ClientName"))
product = spark.createDataFrame([(1, "bicicleta usada"), (2, "caneta Mont Blanc")], ("ProductID", "ProductDescription"))

In [ ]:
transaction.show()

+-------------+---------+--------+----------------+
|TransactionID|ProductID|ClientID|TransactionValue|
+-------------+---------+--------+----------------+
|            1|        1|       1|             200|
|            2|        1|       2|             100|
|            3|        2|       1|             300|
|            4|        2|       2|             500|
|            5|        1|       1|              30|
+-------------+---------+--------+----------------+



In [ ]:
client.show()

+--------+----------+
|ClientID|ClientName|
+--------+----------+
|       1|     Sarah|
|       2|      Joao|
+--------+----------+



In [ ]:
product.show()

+---------+------------------+
|ProductID|ProductDescription|
+---------+------------------+
|        1|   bicicleta usada|
|        2| caneta Mont Blanc|
+---------+------------------+



In [ ]:
df = transaction.join(client, "ClientID", "left")

In [ ]:
df = df.join(product, "ProductID", "left")

In [ ]:
df.show()

+---------+--------+-------------+----------------+----------+------------------+
|ProductID|ClientID|TransactionID|TransactionValue|ClientName|ProductDescription|
+---------+--------+-------------+----------------+----------+------------------+
|        1|       1|            1|             200|     Sarah|   bicicleta usada|
|        1|       1|            5|              30|     Sarah|   bicicleta usada|
|        1|       2|            2|             100|      Joao|   bicicleta usada|
|        2|       1|            3|             300|     Sarah| caneta Mont Blanc|
|        2|       2|            4|             500|      Joao| caneta Mont Blanc|
+---------+--------+-------------+----------------+----------+------------------+



In [ ]:
df = df.select("TransactionValue", "ClientName", "ProductDescription")
df.show()

+----------------+----------+------------------+
|TransactionValue|ClientName|ProductDescription|
+----------------+----------+------------------+
|             200|     Sarah|   bicicleta usada|
|              30|     Sarah|   bicicleta usada|
|             100|      Joao|   bicicleta usada|
|             300|     Sarah| caneta Mont Blanc|
|             500|      Joao| caneta Mont Blanc|
+----------------+----------+------------------+

